# Code


## Importing libraries


In [ ]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
import numpy as np
import os
import base64
import IPython

## Functions

### Technical Functions


In [ ]:
def filter_attributes(data, attributes):
    return [
        {key: entry[key] for key in attributes if key in entry}
        for entry in data
    ]

In [ ]:
def get_personal_details(team_id):
  url = "https://fantasy.premierleague.com/api/entry/"+team_id+"/"
  response = requests.get(url)
  if response.status_code != 200:
    print("Error: Invalid team ID or unable to fetch data.")
    return None
  team_data_json = response.json()
  leagues = team_data_json["leagues"]["classic"]
  current_gameweek = team_data_json["current_event"]
  url = "https://fantasy.premierleague.com/api/entry/"+team_id+"/"
  team_data_json = requests.get(url).json()
  team_data_personal = {key: value for key, value in team_data_json.items() if key in {
      "id","player_first_name","player_last_name","name","years_active","summary_overall_points","summary_overall_rank","summary_event_points"}}

  column_order = [
    "id",
    "player_first_name",
    "player_last_name",
    "name",
    "years_active",
    "summary_overall_points",
    "summary_overall_rank",
    "summary_event_points"
]

# Reorder the DataFrame columns
  team_data_personal = pd.DataFrame(team_data_personal,index=[0])
  team_data_personal = team_data_personal[column_order]
  return team_data_personal,leagues,current_gameweek

In [ ]:
# Gets all the players in a managers team and their gw stats using team id and returns dataframe
def get_managers_players(team_id,current_gameweek):

  # get players in the manager's team
  url = "https://fantasy.premierleague.com/api/entry/"+team_id+"/event/"+str(current_gameweek)+"/picks/"
  manager_players_json = requests.get(url).json()
  attributes_to_keep = ["element","is_captain","multiplier","element_type","is_vice_captain"]
  manager_players = filter_attributes(manager_players_json["picks"],attributes_to_keep)
  managers_players = pd.DataFrame(manager_players)

  # get players data
  url = "https://fantasy.premierleague.com/api/bootstrap-static/"
  players_json = requests.get(url).json()
  players = players_json["elements"]
  attributes_to_keep = ["first_name","id","photo","second_name","team","selected_by_percent","transfers_in","transfers_out"]
  players = filter_attributes(players,attributes_to_keep)
  players = pd.DataFrame(players)
  managers_players_filtered = players[players["id"].isin(managers_players["element"])]
  managers_players = pd.merge(
    managers_players_filtered,  # The filtered players DataFrame
    managers_players,  # The managers_players DataFrame
    left_on="id",      # Column in players DataFrame
    right_on="element" # Column in managers_players DataFrame
  )

    # get gameweek specific data
  url = "https://fantasy.premierleague.com/api/event/"+str(current_gameweek)+"/live/"
  gw_players_json = requests.get(url).json()
  gw_players_json = gw_players_json["elements"]
  filtered_data = [
    {
        'id': player['id'],
        'minutes': player['stats']['minutes'],
        'goals_scored': player['stats']['goals_scored'],
        'assists': player['stats']['assists'],
        'total_points': player['stats']['total_points'],
        'clean_sheets': player['stats']['clean_sheets'],
        'expected_goals': player['stats']['expected_goals'],
        'expected_assists': player['stats']['expected_assists'],
        'expected_goal_involvements': player['stats']['expected_goal_involvements'],
        'expected_goals_conceded' : player['stats']['expected_goals_conceded'],
        'goals_conceded' : player['stats']['goals_conceded']
    }
    for player in gw_players_json if player['id'] in managers_players['element'].values
]

# Create a DataFrame from the filtered data
  filtered_df = pd.DataFrame(filtered_data)
  # merge with dataframe
  managers_players_this_gw = pd.merge(managers_players, filtered_df, left_on="id", right_on="id").drop(columns=["element"])
  column_order = [
    'id',               # Player ID
    'first_name',       # First name
    'second_name',      # Second name
    'photo',            # Photo
    'team',             # Team
    'element_type',     # Player position/type
    'selected_by_percent',  # Percentage selected
    'transfers_in',     # Transfers in
    'transfers_out',    # Transfers out
    'is_captain',       # Is the player a captain
    'is_vice_captain',  # Is the player a vice-captain
    'multiplier',       # Captain multiplier
    'minutes',          # Minutes played
    'goals_scored',     # Goals scored
    'assists',          # Assists
    'total_points',     # Total points
    'clean_sheets',     # Clean sheets
    'expected_goals',   # Expected goals
    'expected_assists', # Expected assists
    'expected_goal_involvements',  # Expected goal involvements
    'expected_goals_conceded',
    'goals_conceded'
]

# Reorder the DataFrame columns
  managers_players_this_gw = managers_players_this_gw[column_order]
  managers_players_this_gw = managers_players_this_gw.astype({'selected_by_percent':'float' , 'transfers_in':'int','transfers_out':'int','multiplier':'int','minutes':'int','goals_scored':'int',
    'assists':'int','total_points':'int','clean_sheets':'int','expected_goals':'float','expected_assists':'float','expected_goal_involvements':'float','expected_goals_conceded':'float','goals_conceded':'float'})

  return managers_players_this_gw

In [ ]:
def select_league_by_name(data):
  leagues_names = []
  leagues_ids = []
  for item in data:
      leagues_names.append(item["name"])
      leagues_ids.append(item["id"])
  print("Please Select a League: \n")
  i = 0
  for league in leagues_names:
    print(str(i+1)+". "+league+"\n")
    i += 1

  input_league = input()
  return leagues_ids[int(input_league)-1]


In [ ]:
def select_league_type(data):
  types = list(data["leagues"].keys())
  types.remove("cup_matches")
  types.remove("cup")
  print("Please Select a League Type: \n")
  i = 0
  for type in types:
    print(str(i+1)+". "+type+"\n")
    i += 1
  league = int(input())-1
  return types[league]

In [ ]:
def get_league_standings(league_id, max_players=5000):
    page = 1
    all_standings = []

    try:
        url = f"https://fantasy.premierleague.com/api/leagues-classic/{league_id}/standings/?page_standings={page}"
        response = requests.get(url)
        if response.status_code != 200:
            print("Invalid league ID or unable to fetch data.")
            return None
        league_data_json = response.json()
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

    while True:
        url = f"https://fantasy.premierleague.com/api/leagues-classic/{league_id}/standings/?page_standings={page}"
        league_data_json = requests.get(url).json()
        league_name = league_data_json['league']['name']
        standings = league_data_json["standings"]["results"]
        attributes_to_keep = ['rank', 'player_name', "event_total", 'entry_name', 'total', "last_rank", "entry"]
        filtered_standings = filter_attributes(standings, attributes_to_keep)
        all_standings.extend(filtered_standings)

        if len(all_standings) > max_players:
            print(f"Error: The number of players exceeds the limit of {max_players}.")
            return None

        if not league_data_json["standings"]["has_next"]:
            break
        page += 1

    standings_df = pd.DataFrame(all_standings)
    standings_df.rename(columns={
        'event_total': 'gameweek_points',
        'total': 'total_points',
        'entry_name': 'team_name'
        ,'entry':'id'
    }, inplace=True)
    standings_df = standings_df[["id", 'rank', 'player_name', 'team_name', "total_points", "gameweek_points", "last_rank"]]

    return standings_df, league_name


In [ ]:
def calculate_personal_stats(personal_info,managers_players):
  # getting the player with the most points
  max_points_indx = managers_players["total_points"].idxmax()
  max_points_player_points = managers_players.loc[max_points_indx,"total_points"]
  max_points_player_name = managers_players.loc[max_points_indx,"first_name"]+" "+managers_players.loc[max_points_indx,"second_name"]
  max_xg_indx = managers_players["expected_goals"].idxmax()
  max_xg_player_name = managers_players.loc[max_xg_indx,"first_name"]+" "+managers_players.loc[max_xg_indx,"second_name"]
  max_xg_player_id = managers_players.loc[max_xg_indx,"id"]
  max_xa_player_indx = managers_players["expected_assists"].idxmax()
  max_xa_player_name = managers_players.loc[max_xa_player_indx,"first_name"]+" "+managers_players.loc[max_xa_player_indx,"second_name"]
  max_xa_player_id = managers_players.loc[max_xa_player_indx,"id"]
  max_xa_player_xa = managers_players.loc[max_xa_player_indx,"expected_assists"]
  max_xg_player_xg = managers_players.loc[max_xg_indx,"expected_goals"]

  team_stats = managers_players.groupby('team', as_index=False).agg({
        'goals_conceded': 'max',  # Assuming max since all players in a team share the same value
        'expected_goals_conceded': 'max'  # Same assumption as above
    })
  total_expected_goal_involvements = managers_players["expected_goal_involvements"].sum()
  total_goal_involvments = managers_players["goals_scored"].sum() +  managers_players["assists"].sum()
  average_goal_involvments = managers_players["goals_scored"].mean() + managers_players["assists"].mean()
  total_goals_conceded = team_stats["goals_conceded"].sum()
  total_expected_goals_conceded = team_stats["expected_goals_conceded"].sum()
  average_expected_goals_conceded = team_stats["expected_goals_conceded"].mean()
  total_expected_goals_conceded = managers_players["expected_goals_conceded"].sum()
  #managers_players = managers_players.astype({'selected_by_percent':'float'})
  average_selection_precentage = managers_players["selected_by_percent"].mean()
  average_xa = managers_players["expected_assists"].mean()
  average_xg = managers_players["expected_goals"].mean()
  total_xa = managers_players["expected_assists"].sum()
  total_xg = managers_players["expected_goals"].sum()

  personal_stats_df = pd.DataFrame({
    'id': personal_info['id'],
    'managers_name':personal_info["player_first_name"]+" "+personal_info["player_last_name"],
    'team_name': personal_info["name"],
    'total_goal_involvments': [total_goal_involvments],
    'average_goal_involvments': [average_goal_involvments],
    'total_expected_goal_involvements': [total_expected_goal_involvements],
    'total_goals_conceded': [total_goals_conceded],
    'average_expected_goals_conceded': [average_expected_goals_conceded],
    'total_expected_goals_conceded': [total_expected_goals_conceded],
    'average_selection_precentage': [average_selection_precentage],
    'average_xa': [average_xa],
    'average_xg': [average_xg],
    'total_xa': [total_xa],
    'total_xg': [total_xg],
    'highest_scoring_player_id': [managers_players.loc[max_points_indx,"id"]],
    'highest_scoring_player_name': [max_points_player_name],
    'highest_scoring_player_points': [max_points_player_points],
    'highest_xa_player_id': [max_xa_player_id],
    'highest_xa_player_xa': [max_xa_player_xa],
    'highest_xa_player_name': [max_xa_player_name],
    'highest_xg_player_id': [max_xg_player_id],
    'highest_xg_player_name': [max_xg_player_name],
    'highest_xg_player_xg': [max_xg_player_xg]
})
  return personal_stats_df

In [ ]:
def calculate_similarity(own_players, opposing_players):
    """
    Calculate the similarity percentage between two managers_players DataFrames
    and identify players present in df1 but not in df2.

    Parameters:
    - df1: DataFrame for the first manager's players
    - df2: DataFrame for the second manager's players

    Returns:
    - similarity_percentage: Percentage of players in df1 also present in df2
    - unique_players_df: DataFrame of players in df1 but not in df2
    """
    df1_ids = set(own_players['id'])
    df2_ids = set(opposing_players['id'])

    common_players = df1_ids.intersection(df2_ids)
    unique_players = df1_ids.difference(df2_ids)

    similarity_percentage = (len(common_players) / len(df1_ids)) * 100 if df1_ids else 0

    unique_players_df = own_players[own_players['id'].isin(unique_players)]

    return round(similarity_percentage,2), unique_players_df


In [ ]:
def get_league_stats(standings, personal_info):
  managers_personal_info = []
  managers_players = []
  manager_stats = []
  max_average_selection_precentage = 0
  min_average_selection_precentage = 100
  max_average_selection_precentage_name = ''
  min_average_selection_precentage_name = ''
  total_xa = 0
  total_xg = 0
  total_expected_goals_conceded = 0
  total_goal_involvments = 0
  num_managers = len(standings)

  for manager_id in standings['id']:
    personal_info_manager, na, current_gameweek = get_personal_details(str(manager_id))
    managers_personal_info.append(personal_info_manager)
    players = get_managers_players(str(manager_id), current_gameweek)
    managers_players.append(players)
    current_manager_stats = calculate_personal_stats(personal_info_manager, players)
    manager_stats.append(current_manager_stats)
    if max_average_selection_precentage < float(current_manager_stats['average_selection_precentage'][0]):
      max_average_selection_precentage = float(current_manager_stats['average_selection_precentage'][0])
      max_average_selection_precentage_name = current_manager_stats['managers_name'][0]
    if min_average_selection_precentage > float(current_manager_stats['average_selection_precentage'][0]):
      min_average_selection_precentage = float(current_manager_stats['average_selection_precentage'][0])
      min_average_selection_precentage_name = current_manager_stats['managers_name'][0]
    total_goal_involvments += current_manager_stats['average_goal_involvments'][0]
    total_expected_goals_conceded += current_manager_stats['total_expected_goals_conceded'][0]
    total_xa += current_manager_stats['average_xa'][0]
    total_xg += current_manager_stats['average_xg'][0]

  average_goal_involvments = total_goal_involvments / num_managers
  total_expected_goals_conceded = total_expected_goals_conceded / num_managers
  average_xa = total_xa / num_managers
  average_xg = total_xg / num_managers

  my_id = int(personal_info['id'][0])
  my_index = standings.loc[standings['id'] == my_id].index[0]
  if my_index == len(standings) - 1:
    below_player_name = None
    points_away_from_below = None
    below_player_id = None
  else:
    below_index = my_index + 1
    below_player = standings.loc[below_index]
    below_player_name = below_player['player_name']
    points_away_from_below = int(personal_info['summary_overall_points'][0]) - int(below_player['total_points'])
    below_player_id = [standings.loc[standings['player_name'] == below_player_name, 'id'].iloc[0]]
  if my_index == 0:
    above_player_name = None
    points_away_from_above = None
    points_away_from_top = None
    above_player_id = None
  else:
    above_index = my_index - 1
    above_player = standings.loc[above_index]
    above_player_name = above_player['player_name']
    points_away_from_above = int(above_player['total_points']) - int(personal_info['summary_overall_points'][0])
    points_away_from_top = int(standings['total_points'].max()) - int(personal_info['summary_overall_points'][0])
    above_player_id =  [standings.loc[standings['player_name'] == above_player_name, 'id'].iloc[0]]

  average_gameweek_points = int(standings['gameweek_points'].mean())
  max_average_selection_precentage = round(max_average_selection_precentage, 2)
  min_average_selection_precentage = round(min_average_selection_precentage, 2)
  top_player_this_gameweek = standings.loc[standings['gameweek_points'].idxmax()]
  top_player_name = top_player_this_gameweek['player_name']
  top_player_points = top_player_this_gameweek['gameweek_points']

  overall_league_stats = pd.DataFrame({
      'average_gameweek_points': [average_gameweek_points],
      'top_player_this_gameweek_name': [top_player_name],
      'top_player_this_gameweek_points': [top_player_points],
      'max_average_selection_precentage_name': [max_average_selection_precentage_name],
      'max_average_selection_precentage': [max_average_selection_precentage],
      'min_average_selection_precentage_name': [min_average_selection_precentage_name],
      'min_average_selection_precentage': [min_average_selection_precentage],
      'average_goal_involvments': [average_goal_involvments],
      'average_expected_goals_conceded': [total_expected_goals_conceded],
      'average_xa': [average_xa],
      'average_xg': [average_xg]
  })

  personal_league_stats = pd.DataFrame({
      'below_player_name': [below_player_name],
      'below_player_id': below_player_id,
      'points_away_from_below': [points_away_from_below],
      'above_player_name': [above_player_name],
      'above_player_id': above_player_id,
      'points_away_from_above': [points_away_from_above],
      'points_away_from_top': [points_away_from_top]
  })

  return overall_league_stats, personal_league_stats


### Chart Funcions

In [ ]:


def plot_clustered_chart(personal_stats_df, league_stats_df, output_path='/content/'):
    # Metrics excluding 'Expected Goals Conceded'
    labels = ['XG', 'XA', 'Goal Involvements']

    personal_values = [
        personal_stats_df['average_xg'].iloc[0],
        personal_stats_df['average_xa'].iloc[0],
        personal_stats_df['average_goal_involvments'].iloc[0]
    ]

    league_values = [
        league_stats_df['average_xg'].iloc[0],
        league_stats_df['average_xa'].iloc[0],
        league_stats_df['average_goal_involvments'].iloc[0]
    ]

    x = np.arange(len(labels))  # The label locations
    width = 0.35  # The width of the bars

    # Plot for the first set of metrics
    fig, ax = plt.subplots(figsize=(10, 6))

    rects1 = ax.bar(x - width/2, personal_values, width, label='Personal Stats', color='blue')
    rects2 = ax.bar(x + width/2, league_values, width, label='League Average', color='green')

    ax.set_xlabel('Stats')
    ax.set_title('Personal Stats vs League Average (Per Player)')
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend()

    # Add values on top of the bars
    def add_values(rects):
        for rect in rects:
            height = rect.get_height()  # Access individual bar height
            ax.annotate(f'{round(height, 2)}',
                        xy=(rect.get_x() + rect.get_width() / 2, height),
                        xytext=(0, 3),  # 3 points vertical offset
                        textcoords="offset points",
                        ha='center', va='bottom', fontsize=10)

    # Accessing the actual bars from BarContainer object
    add_values(rects1.get_children())
    add_values(rects2.get_children())

    # Save the figure as an image file
    plt.tight_layout()
    image_path = os.path.join(output_path, "stats_chart.png")
    plt.savefig(image_path)
    plt.close()  # Close the plot to free memory

    return image_path

In [ ]:
def create_venn_diagram_from_dfs(df1, df2, label1="Your Team", label2="His Team", number='1',key_column="id", output_path='/content/'):
    """
    Creates a Venn diagram comparing two DataFrames based on a specified key column and displays player names.

    Parameters:
    - df1 (DataFrame): The first DataFrame containing player data.
    - df2 (DataFrame): The second DataFrame containing player data.
    - player_id (str): A unique identifier for the player or comparison context.
    - output_path (str): Path to save the generated Venn diagram image.
    - label1 (str): Label for the first DataFrame in the Venn diagram.
    - label2 (str): Label for the second DataFrame in the Venn diagram.
    - key_column (str): Column name used to compare players (e.g., 'id').

    Returns:
    - str: The path to the saved Venn diagram image.
    """
    # Extract sets of the key column
    set1 = set(df1[key_column])
    set2 = set(df2[key_column])

    # Extract player names
    names1 = df1.set_index(key_column)['first_name'] + " " + df1.set_index(key_column)['second_name']
    names2 = df2.set_index(key_column)['first_name'] + " " + df2.set_index(key_column)['second_name']

    # Players in each subset
    only_in_set1 = [names1[id] for id in set1 - set2 if id in names1]
    only_in_set2 = [names2[id] for id in set2 - set1 if id in names2]
    in_both_sets = [names1[id] for id in set1 & set2 if id in names1]

    # Create the Venn diagram
    fig, ax = plt.subplots(figsize=(10, 8))
    venn = venn2([set1, set2], (label1, label2))

    # Add player names to each subset
    def annotate_subset(subset_id, names):
        subset = venn.get_label_by_id(subset_id)
        if subset:
            subset.set_text("\n".join(names))

    annotate_subset('10', only_in_set1)  # Only in set1
    annotate_subset('01', only_in_set2)  # Only in set2
    annotate_subset('11', in_both_sets)  # In both sets

    # Customize colors and transparency
    venn.get_patch_by_id('10').set_color('#FF9999')  # Only in df1
    venn.get_patch_by_id('01').set_color('#9999FF')  # Only in df2
    venn.get_patch_by_id('11').set_color('#99FF99')  # In both df1 and df2
    venn.get_patch_by_id('10').set_alpha(0.7)
    venn.get_patch_by_id('01').set_alpha(0.7)
    venn.get_patch_by_id('11').set_alpha(0.7)

    # Add title
    ax.set_title(f"Comparison of {label1} and {label2} Players")

    # Save the figure as an image file
    plt.tight_layout()
    image_path = os.path.join(output_path, "venn_diagram"+number+".png")
    plt.savefig(image_path, format="png", bbox_inches="tight")
    plt.close()  # Close the plot to free memory

    return image_path

In [ ]:
def plot_comparison_chart(personal_stats_df, output_path='/content/'):
    labels = ['Goals Conceded', 'Goal Involvments']
    personal_values = [
        personal_stats_df['total_expected_goals_conceded'].iloc[0],
        personal_stats_df['total_expected_goal_involvements'].iloc[0]
    ]
    actual_values = [
        personal_stats_df['total_goals_conceded'].iloc[0],
        personal_stats_df['total_goal_involvments'].iloc[0]
    ]
    x = np.arange(len(labels))
    width = 0.35
    fig, ax = plt.subplots(figsize=(10, 6))
    rects1 = ax.bar(x - width/2, personal_values, width, label='Expected Stats', color='blue')
    rects2 = ax.bar(x + width/2, actual_values, width, label='Actual Stats', color='red')
    ax.set_xlabel('Stats')
    ax.set_title(f'Comparison of Expected vs Actual values')
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend()

    def add_values(rects):
        for rect in rects:
            height = rect.get_height()
            ax.annotate(f'{round(height, 2)}',
                        xy=(rect.get_x() + rect.get_width() / 2, height),
                        xytext=(0, 3),
                        textcoords="offset points",
                        ha='center', va='bottom', fontsize=10)

    add_values(rects1)
    add_values(rects2)

    plt.tight_layout()
    image_path = os.path.join(output_path, "comparison_chart.png")
    plt.savefig(image_path)
    plt.close()

    return image_path

In [ ]:
def image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

### Formulating Email Function

In [ ]:
def generate_fpl_cruncher_top_html(team_id, league_id, game_week,output_path='output.html'):
    personal_info, leagues, current_gameweek = get_personal_details(team_id)
    current_gameweek = game_week
    managers_players = get_managers_players(team_id, current_gameweek)
    recipient_name = personal_info['player_first_name'][0] + " " + personal_info['player_last_name'][0]
    gameweek_points = personal_info['summary_event_points'][0]
    standings, league_name = get_league_standings(league_id)
    overall_league_stats, personal_league_stats = get_league_stats(standings, personal_info)
    personal_stats = calculate_personal_stats(personal_info, managers_players)
    below_player_players = get_managers_players(str(team_id), current_gameweek)
    league_average_points = overall_league_stats['average_gameweek_points'][0]
    highest_scoring_player_name = personal_stats['highest_scoring_player_name'][0]
    highest_scoring_player_points = personal_stats['highest_scoring_player_points'][0]
    highest_scoring_player_img_id = managers_players[managers_players['id'] == personal_stats['highest_scoring_player_id'][0]]['photo'].iloc[0]
    highest_scoring_player_img_id = highest_scoring_player_img_id.split('.')[0]
    highest_xg_player_name = personal_stats['highest_xg_player_name'][0]
    highest_xg_player_value = personal_stats['highest_xg_player_xg'][0]
    highest_xg_player_img_id = managers_players[managers_players['id'] == personal_stats['highest_xg_player_id'][0]]['photo'].iloc[0]
    highest_xg_player_img_id = highest_xg_player_img_id.split('.')[0]
    highest_xa_player_name = personal_stats['highest_xa_player_name'][0]
    highest_xa_player_value = personal_stats['highest_xa_player_xa'][0]
    highest_xa_player_img_id = managers_players[managers_players['id'] == personal_stats['highest_xa_player_id'][0]]['photo'].iloc[0]
    highest_xa_player_img_id = highest_xa_player_img_id.split('.')[0]
    player_below_name = personal_league_stats['below_player_name'][0]
    points_ahead_of_player_below = personal_league_stats['points_away_from_below'][0]
    highest_selection_rate_player_name = overall_league_stats['max_average_selection_precentage_name'][0]
    highest_selection_rate_value = overall_league_stats['max_average_selection_precentage'][0]
    lowest_selection_rate_player_name = overall_league_stats['min_average_selection_precentage_name'][0]
    lowest_selection_rate_value = overall_league_stats['min_average_selection_precentage'][0]
    below_player_id = personal_league_stats['below_player_id'][0]
    below_player_players = get_managers_players(str(below_player_id), current_gameweek)
    similarity_percentage_below = round(calculate_similarity(managers_players, below_player_players)[0], 2)
    venn_image_path_below = create_venn_diagram_from_dfs(managers_players, below_player_players, recipient_name, player_below_name)
    expectedVsactual_image_path = plot_comparison_chart(personal_stats)
    clustered_image_path = plot_clustered_chart(personal_stats, overall_league_stats)

    # Convert images to base64
    venn_image_base64_below = image_to_base64(venn_image_path_below)
    expectedVsactual_base64 = image_to_base64(expectedVsactual_image_path)
    clustered_image_base64 = image_to_base64(clustered_image_path)

    html_content = f"""<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>FPL Cruncher</title>
  <style>
    body {{
      margin: 0;
      padding: 0;
      font-family: Arial, sans-serif;
      background-color: black;
      color: white;
    }}
    .header-image {{
      display: block;
      margin: 0 auto;
      width: 100%;
      height: 50vh; /* Header occupies half the viewport height */
      object-fit: fill; /* Ensures the image covers the header area */
    }}
    .content {{
      padding: 30px;
      text-align: left;
      color: white; /* Ensures paragraph text is white */
      font-size: 18px;
      background-color: #050507;
    }}
    .footer {{
      background-color: #f1f1f1;
      color: #555;
      text-align: center;
      padding: 20px;
      font-size: 14px;
    }}
      .graph-container {{
    text-align: center;
    margin: 20px auto;
  }}
  .graph-container img {{
    width: 60%;
    height: auto;
    margin: 20px auto;
  }}
   @media (max-width: 768px) {{
    .graph-container img {{
      width: 100%; /* Increase size on smaller screens */
    }}
  </style>
</head>
<body>
  <table style="border-spacing: 0; width: 100%; border: none;">
    <tr>
      <td>
        <div style="width: 100%; margin: 0 auto; border: none; padding: 0;">
<img src="https://i.ibb.co/M6rF5HL/FPL-CRUNCHER.png" alt="FPL-CRUNCHER" width="1580" height="518" data-is360="0" data-load="full" class="" style="width: 100%;height: auto;display: block;">
          <div class="content">
            <h2 style="color: #ade658; font-size: 40px;">Hello, {recipient_name}</h2>
            <p>Welcome to FPL CRUNCHER!</p>
            <p>This game week you scored <strong>{gameweek_points}</strong> points. The average points in the <strong>{league_name}</strong> league this week were <strong>{league_average_points}</strong> points. Keep up the fantastic work!</p>
            <p>Your highest scoring player this week was <strong>{highest_scoring_player_name}</strong> with <strong>{highest_scoring_player_points}</strong> points.</p>
            <div style="text-align: center; margin: 20px 0;">
              <img src="https://resources.premierleague.com/premierleague/photos/players/110x140/p{highest_scoring_player_img_id}.png"
                   alt="{highest_scoring_player_name}" style="width: 150px; height: 190px; margin-top: 10px;">
            </div>
            <p>Your player with the highest xG this week was <strong>{highest_xg_player_name}</strong> with an xG of <strong>{highest_xg_player_value}</strong>.</p>
            <div style="text-align: center; margin: 20px 0;">
              <img src="https://resources.premierleague.com/premierleague/photos/players/110x140/p{highest_xg_player_img_id}.png"
                   alt="{highest_xg_player_name}" style="width: 150px; height: 190px; margin-top: 10px;">
            </div>
            <p>Your player with the highest xA this week was <strong>{highest_xa_player_name}</strong> with an xA of <strong>{highest_xa_player_value}</strong>.</p>
            <div style="text-align: center; margin: 20px 0;">
              <img src="https://resources.premierleague.com/premierleague/photos/players/110x140/p{highest_xa_player_img_id}.png"
                   alt="{highest_xa_player_name}" style="width: 150px; height: 190px; margin-top: 10px;">
            </div>
            <p>Let's see if your team is over or under performing:</p>
            <div class="graph-container">
            <img src="data:image/png;base64,{expectedVsactual_base64}"
                 alt="Expected Goal involvements and goals conceded vs actual">
                 </div>
            <h3 style="color: #ade658; font-size: 32px; margin-bottom: 20px;">League Insights</h3>
            <p>Here are some stats about your team compared to the <strong>{league_name}</strong> average:</p>
             <div class="graph-container">
            <img src="data:image/png;base64,{clustered_image_base64}"
                 alt="Team vs League Stats Comparison">
                 </div>
            <p>Congrats!! you are currently the top player in  <strong>{league_name}</strong>. Does it feel lonely at the top? ;)</p>
            <p>You are directly ahead of <strong>{player_below_name}</strong> with <strong>{points_ahead_of_player_below}</strong> points</p>
            <p><strong>{highest_selection_rate_player_name}</strong> seems like the one with the most template team in the league with an average selection rate of <strong>{highest_selection_rate_value}%</strong>.</p>
            <p>While <strong>{lowest_selection_rate_player_name}</strong> loves going bold and differential with an average selection rate of <strong>{lowest_selection_rate_value}%</strong>.</p>
            <h3 style="color: #ade658; font-size: 32px; margin-bottom: 20px;">Your Team vs Directly Below Player</h3>
            <h4 style="color: #ade658;">Comparison with <strong>{player_below_name}</strong></h4>
            <p>Your teams are <strong>{similarity_percentage_below}%</strong> simmilar.</p>
             <div class="graph-container">
            <img src="data:image/png;base64,{venn_image_base64_below}"
                 alt="Venn Diagram for {player_below_name}">
                 </div>
            <p style="text-align: center;">Stay ahead in your Fantasy Premier League journey with weekly stats, insights, and performance reviews tailored just for you!</p>
          </div>
          <div class="footer">
            <p>FPL CRUNCHER  created by Adham Ebaid</p>
          </div>
        </div>
      </td>
    </tr>
  </table>
</body>
</html>

"""

    with open(output_path, 'w') as file:
        file.write(html_content)
    return output_path

In [ ]:
def generate_fpl_cruncher_normal_html(team_id, league_id,game_week,output_path='output.html'):
    personal_info, leagues, current_gameweek = get_personal_details(team_id)
    current_gameweek = game_week
    managers_players = get_managers_players(team_id, current_gameweek)

    recipient_name = personal_info['player_first_name'][0] + " " + personal_info['player_last_name'][0]
    gameweek_points = personal_info['summary_event_points'][0]
    standings, league_name = get_league_standings(league_id)
    overall_league_stats, personal_league_stats = get_league_stats(standings, personal_info)
    personal_stats = calculate_personal_stats(personal_info, managers_players)

    below_player_players = get_managers_players(str(team_id), current_gameweek)
    league_average_points = overall_league_stats['average_gameweek_points'][0]
    highest_scoring_player_name = personal_stats['highest_scoring_player_name'][0]
    highest_scoring_player_points = personal_stats['highest_scoring_player_points'][0]
    highest_scoring_player_img_id = managers_players[managers_players['id'] == personal_stats['highest_scoring_player_id'][0]]['photo'].iloc[0]
    highest_scoring_player_img_id = highest_scoring_player_img_id.split('.')[0]
    highest_xg_player_name = personal_stats['highest_xg_player_name'][0]
    highest_xg_player_value = personal_stats['highest_xg_player_xg'][0]
    highest_xg_player_img_id = managers_players[managers_players['id'] == personal_stats['highest_xg_player_id'][0]]['photo'].iloc[0]
    highest_xg_player_img_id = highest_xg_player_img_id.split('.')[0]
    highest_xa_player_name = personal_stats['highest_xa_player_name'][0]
    highest_xa_player_value = personal_stats['highest_xa_player_xa'][0]
    highest_xa_player_img_id = managers_players[managers_players['id'] == personal_stats['highest_xa_player_id'][0]]['photo'].iloc[0]
    highest_xa_player_img_id = highest_xa_player_img_id.split('.')[0]
    points_behind_top_player = personal_league_stats['points_away_from_top'][0]
    top_player_name = standings['player_name'][0]
    player_below_name = personal_league_stats['below_player_name'][0]
    points_ahead_of_player_below = personal_league_stats['points_away_from_below'][0]
    player_above_name = personal_league_stats['above_player_name'][0]
    points_behind_player_above = personal_league_stats['points_away_from_above'][0]
    highest_selection_rate_player_name = overall_league_stats['max_average_selection_precentage_name'][0]
    highest_selection_rate_value = overall_league_stats['max_average_selection_precentage'][0]
    lowest_selection_rate_player_name = overall_league_stats['min_average_selection_precentage_name'][0]
    lowest_selection_rate_value = overall_league_stats['min_average_selection_precentage'][0]
    below_player_id = personal_league_stats['below_player_id'][0]
    above_player_id = personal_league_stats['above_player_id'][0]
    below_player_players = get_managers_players(str(below_player_id), current_gameweek)
    above_player_players = get_managers_players(str(above_player_id), current_gameweek)
    similarity_percentage_above = round(calculate_similarity(managers_players, above_player_players)[0], 2)
    venn_image_path_above = create_venn_diagram_from_dfs(managers_players, above_player_players, recipient_name, player_above_name,'1')
    similarity_percentage_below = round(calculate_similarity(managers_players, below_player_players)[0], 2)
    venn_image_path_below = create_venn_diagram_from_dfs(managers_players, below_player_players, recipient_name, player_below_name,'2')
    expectedVsactual_image_path = plot_comparison_chart(personal_stats)
    clustered_image_path = plot_clustered_chart(personal_stats, overall_league_stats)

    # Convert images to base64
    venn_image_base64_above = image_to_base64(venn_image_path_above)
    venn_image_base64_below = image_to_base64(venn_image_path_below)
    expectedVsactual_base64 = image_to_base64(expectedVsactual_image_path)
    clustered_image_base64 = image_to_base64(clustered_image_path)

    html_content = f"""<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>FPL Cruncher</title>
  <style>
    body {{
      margin: 0;
      padding: 0;
      font-family: Arial, sans-serif;
      background-color: black;
      color: white;
    }}
    .header-image {{
      display: block;
      margin: 0 auto;
      width: 100%;
      height: 50vh; /* Header occupies half the viewport height */
      object-fit: fill; /* Ensures the image covers the header area */
    }}
    .content {{
      padding: 30px;
      text-align: left;
      color: white; /* Ensures paragraph text is white */
      font-size: 18px;
      background-color: #050507;
    }}
    .footer {{
      background-color: #f1f1f1;
      color: #555;
      text-align: center;
      padding: 20px;
      font-size: 14px;
    }}
      .graph-container {{
    text-align: center;
    margin: 20px auto;
  }}
  .graph-container img {{
    width: 60%;
    height: auto;
    margin: 20px auto;
  }}
   @media (max-width: 768px) {{
    .graph-container img {{
      width: 100%; /* Increase size on smaller screens */
    }}
  </style>
</head>
<body>
  <table style="border-spacing: 0; width: 100%; border: none;">
    <tr>
      <td>
        <div style="width: 100%; margin: 0 auto; border: none; padding: 0;">
<img src="https://i.ibb.co/M6rF5HL/FPL-CRUNCHER.png" alt="FPL-CRUNCHER" width="1580" height="518" data-is360="0" data-load="full" class="" style="width: 100%;height: auto;display: block;">
          <div class="content">
            <h2 style="color: #ade658; font-size: 40px;">Hello, {recipient_name}</h2>
            <p>Welcome to FPL CRUNCHER!</p>
            <p>This game week you scored <strong>{gameweek_points}</strong> points. The average points in the <strong>{league_name}</strong> league this week were <strong>{league_average_points}</strong> points. Keep up the fantastic work!</p>
            <p>Your highest scoring player this week was <strong>{highest_scoring_player_name}</strong> with <strong>{highest_scoring_player_points}</strong> points.</p>
            <div style="text-align: center; margin: 20px 0;">
              <img src="https://resources.premierleague.com/premierleague/photos/players/110x140/p{highest_scoring_player_img_id}.png"
                   alt="{highest_scoring_player_name}" style="width: 150px; height: 190px; margin-top: 10px;">
            </div>
            <p>Your player with the highest xG this week was <strong>{highest_xg_player_name}</strong> with an xG of <strong>{highest_xg_player_value}</strong>.</p>
            <div style="text-align: center; margin: 20px 0;">
              <img src="https://resources.premierleague.com/premierleague/photos/players/110x140/p{highest_xg_player_img_id}.png"
                   alt="{highest_xg_player_name}" style="width: 150px; height: 190px; margin-top: 10px;">
            </div>
            <p>Your player with the highest xA this week was <strong>{highest_xa_player_name}</strong> with an xA of <strong>{highest_xa_player_value}</strong>.</p>
            <div style="text-align: center; margin: 20px 0;">
              <img src="https://resources.premierleague.com/premierleague/photos/players/110x140/p{highest_xa_player_img_id}.png"
                   alt="{highest_xa_player_name}" style="width: 150px; height: 190px; margin-top: 10px;">
            </div>
            <p>Let's see if your team is over or under performing:</p>
            <div class="graph-container">
            <img src="data:image/png;base64,{expectedVsactual_base64}"
                 alt="Expected Goal involvements and goals conceded vs actual">
                 </div>
            <h3 style="color: #ade658; font-size: 32px; margin-bottom: 20px;">League Insights</h3>
            <p>Here are some stats about your team compared to the <strong>{league_name}</strong> average:</p>
             <div class="graph-container">
            <img src="data:image/png;base64,{clustered_image_base64}"
                 alt="Team vs League Stats Comparison">
                 </div>
            <p>You are <strong>{points_behind_top_player}</strong> points behind the top player, <strong>{top_player_name}</strong>.</p>
            <p>You are directly ahead of <strong>{player_below_name}</strong> with <strong>{points_ahead_of_player_below}</strong> points, and directly behind <strong>{player_above_name}</strong> with <strong>{points_behind_player_above}</strong> points.</p>
            <p><strong>{highest_selection_rate_player_name}</strong> seems like the one with the most template team in the league with an average selection rate of <strong>{highest_selection_rate_value}%</strong>.</p>
            <p>While <strong>{lowest_selection_rate_player_name}</strong> loves going bold and differential with an average selection rate of <strong>{lowest_selection_rate_value}%</strong>.</p>
            <h3 style="color: #ade658; font-size: 32px; margin-bottom: 20px;">Your Team vs Directly Above and Below Players</h3>
            <h4 style="color: #ade658;">Comparison with <strong>{player_above_name}</strong></h4>
            <p>Your teams are <strong>{similarity_percentage_above}%</strong> simmilar.</p>
             <div class="graph-container">
            <img src="data:image/png;base64,{venn_image_base64_above}"
                 alt="Venn Diagram for {player_above_name}">
                 </div>
            <h4 style="color: #ade658;">Comparison with <strong>{player_below_name}</strong></h4>
            <p>Your teams are <strong>{similarity_percentage_below}%</strong> simmilar.</p>
             <div class="graph-container">
            <img src="data:image/png;base64,{venn_image_base64_below}"
                 alt="Venn Diagram for {player_below_name}">
                 </div>
            <p style="text-align: center;">Stay ahead in your Fantasy Premier League journey with weekly stats, insights, and performance reviews tailored just for you!</p>
          </div>
          <div class="footer">
            <p>FPL CRUNCHER  created by Adham Ebaid</p>
          </div>
        </div>
      </td>
    </tr>
  </table>
</body>
</html>

"""

    with open(output_path, 'w') as file:
        file.write(html_content)
    return output_path

In [ ]:
def generate_fpl_cruncher_bottom_html(team_id, league_id,game_week,output_path='output.html'):


    current_gameweek = game_week
    managers_players = get_managers_players(team_id, current_gameweek)
    personal_info, leagues, current_gameweek = get_personal_details(team_id)

    recipient_name = personal_info['player_first_name'][0] + " " + personal_info['player_last_name'][0]
    gameweek_points = personal_info['summary_event_points'][0]
    standings, league_name = get_league_standings(league_id)
    overall_league_stats, personal_league_stats = get_league_stats(standings, personal_info)
    personal_stats = calculate_personal_stats(personal_info, managers_players)

    league_average_points = overall_league_stats['average_gameweek_points'][0]
    highest_scoring_player_name = personal_stats['highest_scoring_player_name'][0]
    highest_scoring_player_points = personal_stats['highest_scoring_player_points'][0]
    highest_scoring_player_img_id = managers_players[managers_players['id'] == personal_stats['highest_scoring_player_id'][0]]['photo'].iloc[0]
    highest_scoring_player_img_id = highest_scoring_player_img_id.split('.')[0]
    highest_xg_player_name = personal_stats['highest_xg_player_name'][0]
    highest_xg_player_value = personal_stats['highest_xg_player_xg'][0]
    highest_xg_player_img_id = managers_players[managers_players['id'] == personal_stats['highest_xg_player_id'][0]]['photo'].iloc[0]
    highest_xg_player_img_id = highest_xg_player_img_id.split('.')[0]
    highest_xa_player_name = personal_stats['highest_xa_player_name'][0]
    highest_xa_player_value = personal_stats['highest_xa_player_xa'][0]
    highest_xa_player_img_id = managers_players[managers_players['id'] == personal_stats['highest_xa_player_id'][0]]['photo'].iloc[0]
    highest_xa_player_img_id = highest_xa_player_img_id.split('.')[0]
    points_behind_top_player = personal_league_stats['points_away_from_top'][0]
    top_player_name = standings['player_name'][0]
    player_above_name = personal_league_stats['above_player_name'][0]
    points_behind_player_above = personal_league_stats['points_away_from_above'][0]
    highest_selection_rate_player_name = overall_league_stats['max_average_selection_precentage_name'][0]
    highest_selection_rate_value = overall_league_stats['max_average_selection_precentage'][0]
    lowest_selection_rate_player_name = overall_league_stats['min_average_selection_precentage_name'][0]
    lowest_selection_rate_value = overall_league_stats['min_average_selection_precentage'][0]
    above_player_id = personal_league_stats['above_player_id'][0]
    above_player_players = get_managers_players(str(above_player_id), current_gameweek)

    similarity_percentage_above = round(calculate_similarity(managers_players, above_player_players)[0], 2)
    venn_image_path_above = create_venn_diagram_from_dfs(managers_players, above_player_players, recipient_name, player_above_name)
    expectedVsactual_image_path = plot_comparison_chart(personal_stats)
    clustered_image_path = plot_clustered_chart(personal_stats, overall_league_stats)

    # Convert images to base64
    venn_image_base64_above = image_to_base64(venn_image_path_above)
    expectedVsactual_base64 = image_to_base64(expectedVsactual_image_path)
    clustered_image_base64 = image_to_base64(clustered_image_path)

    html_content = f"""<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>FPL Cruncher</title>
  <style>
    body {{
      margin: 0;
      padding: 0;
      font-family: Arial, sans-serif;
      background-color: black;
      color: white;
    }}
    .header-image {{
      display: block;
      margin: 0 auto;
      width: 100%;
      height: 50vh; /* Header occupies half the viewport height */
      object-fit: fill; /* Ensures the image covers the header area */
    }}
    .content {{
      padding: 30px;
      text-align: left;
      color: white; /* Ensures paragraph text is white */
      font-size: 18px;
      background-color: #050507;
    }}
    .footer {{
      background-color: #f1f1f1;
      color: #555;
      text-align: center;
      padding: 20px;
      font-size: 14px;
    }}
      .graph-container {{
    text-align: center;
    margin: 20px auto;
  }}
  .graph-container img {{
    width: 60%;
    height: auto;
    margin: 20px auto;
  }}
   @media (max-width: 768px) {{
    .graph-container img {{
      width: 100%; /* Increase size on smaller screens */
    }}
  </style>
</head>
<body>
  <table style="border-spacing: 0; width: 100%; border: none;">
    <tr>
      <td>
        <div style="width: 100%; margin: 0 auto; border: none; padding: 0;">
<img src="https://i.ibb.co/M6rF5HL/FPL-CRUNCHER.png" alt="FPL-CRUNCHER" width="1580" height="518" data-is360="0" data-load="full" class="" style="width: 100%;height: auto;display: block;">
          <div class="content">
            <h2 style="color: #ade658; font-size: 40px;">Hello, {recipient_name}</h2>
            <p>Welcome to FPL CRUNCHER!</p>
            <p>This game week you scored <strong>{gameweek_points}</strong> points. The average points in the <strong>{league_name}</strong> league this week were <strong>{league_average_points}</strong> points. Keep up the fantastic work!</p>
            <p>Your highest scoring player this week was <strong>{highest_scoring_player_name}</strong> with <strong>{highest_scoring_player_points}</strong> points.</p>
            <div style="text-align: center; margin: 20px 0;">
              <img src="https://resources.premierleague.com/premierleague/photos/players/110x140/p{highest_scoring_player_img_id}.png"
                   alt="{highest_scoring_player_name}" style="width: 150px; height: 190px; margin-top: 10px;">
            </div>
            <p>Your player with the highest xG this week was <strong>{highest_xg_player_name}</strong> with an xG of <strong>{highest_xg_player_value}</strong>.</p>
            <div style="text-align: center; margin: 20px 0;">
              <img src="https://resources.premierleague.com/premierleague/photos/players/110x140/p{highest_xg_player_img_id}.png"
                   alt="{highest_xg_player_name}" style="width: 150px; height: 190px; margin-top: 10px;">
            </div>
            <p>Your player with the highest xA this week was <strong>{highest_xa_player_name}</strong> with an xA of <strong>{highest_xa_player_value}</strong>.</p>
            <div style="text-align: center; margin: 20px 0;">
              <img src="https://resources.premierleague.com/premierleague/photos/players/110x140/p{highest_xa_player_img_id}.png"
                   alt="{highest_xa_player_name}" style="width: 150px; height: 190px; margin-top: 10px;">
            </div>
            <p>Let's see if your team is over or under performing:</p>
            <div class="graph-container">
            <img src="data:image/png;base64,{expectedVsactual_base64}"
                 alt="Expected Goal involvements and goals conceded vs actual">
                 </div>
            <h3 style="color: #ade658; font-size: 32px; margin-bottom: 20px;">League Insights</h3>
            <p>Here are some stats about your team compared to the <strong>{league_name}</strong> average:</p>
             <div class="graph-container">
            <img src="data:image/png;base64,{clustered_image_base64}"
                 alt="Team vs League Stats Comparison">
                 </div>
                 <p>Looks like you are the bottom player in <strong>{league_name}</strong> Yikes.... Hopefully this changes soon.</p>
            <p>You are <strong>{points_behind_top_player}</strong> points behind the top player, <strong>{top_player_name}</strong>.</p>
            <p>You are directly behind <strong>{player_above_name}</strong> with <strong>{points_behind_player_above}</strong> points.</p>
            <p><strong>{highest_selection_rate_player_name}</strong> seems like the one with the most template team in the league with an average selection rate of <strong>{highest_selection_rate_value}%</strong>.</p>
            <p>While <strong>{lowest_selection_rate_player_name}</strong> loves going bold and differential with an average selection rate of <strong>{lowest_selection_rate_value}%</strong>.</p>
            <h3 style="color: #ade658; font-size: 32px; margin-bottom: 20px;">Your Team vs Directly Above Player</h3>
            <h4 style="color: #ade658;">Comparison with <strong>{player_above_name}</strong></h4>
            <p>Your teams are <strong>{similarity_percentage_above}%</strong> simmilar.</p>
             <div class="graph-container">
            <img src="data:image/png;base64,{venn_image_base64_above}"
                 alt="Venn Diagram for {player_above_name}">
                 </div>
                 </div>
            <p style="text-align: center;">Stay ahead in your Fantasy Premier League journey with weekly stats, insights, and performance reviews tailored just for you!</p>
          </div>
          <div class="footer">
            <p>FPL CRUNCHER  created by Adham Ebaid</p>
          </div>
        </div>
      </td>
    </tr>
  </table>
</body>
</html>

"""

    with open(output_path, 'w') as file:
        file.write(html_content)
    return output_path

In [ ]:
def generate_output_html():
  print("Welcome to FPL Cruncher!!")
  team_id = input("Team ID: ")
  personal_info, leagues, current_gameweek = get_personal_details(team_id)
  league_id = input("League ID: ")
  gw = input("do you want the current gameweek?: \n enter y for yes and n for no: ")
  if gw == 'n' or gw == 'N':
    game_week = input("enter gameweek: ")
  else:
    game_week = current_gameweek
  print("Loading......")
  standings,lg= get_league_standings(league_id)
  my_id = int(personal_info['id'][0])
  my_index = standings.loc[standings['id'] == my_id].index[0]
  if my_index == 0:
    output_path = generate_fpl_cruncher_top_html(team_id, league_id,game_week)
  elif my_index == len(standings) - 1:
    output_path = generate_fpl_cruncher_bottom_html(team_id, league_id,game_week)
  else:
    output_path = generate_fpl_cruncher_normal_html(team_id, league_id,game_week)
  return output_path

# For User

In [ ]:
path = generate_output_html()

In [ ]:
IPython.display.HTML(path)